## word2vec하기

In [39]:
morphs =[[]]
with open('data/for_word2vec.txt') as f:
    for i in range(0,115071):
        for line in f:
            morphs[i].append(line.split(','))
morphs=morphs[0]

for i in range(0,len(morphs)):
    morphs[i][-1] = morphs[i][-1].rstrip('\n')
#morphs

In [40]:
from gensim.models import Word2Vec
model_tag = Word2Vec.load('data/play_w2v.model')

In [41]:
a = model_tag.predict_output_word(['비','오는날'])
a

[('오', 0.011882432),
 ('우산', 0.0049220147),
 ('봄비', 0.0044976664),
 ('비올때', 0.00425057),
 ('비오는날듣기좋은노래', 0.0030199124),
 ('겨울비', 0.0030025754),
 ('비내리', 0.0028487341),
 ('비오는날엔', 0.0022130224),
 ('거리', 0.0021183244),
 ('비내리는날', 0.0018565512)]

In [42]:
import pandas as pd

train_data = pd.read_json('data/train.json', encoding='UTF-8', typ = 'frame')
#train_data

### unique한 tag 목록 만들기

In [43]:
import itertools
uniq_tag = list(set(itertools.chain(*train_data.tags)))

### 영어 소문자로

for i in range(0,len(uniq_tag)):
    uniq_tag[i] = uniq_tag[i].lower()
uniq_tag

['',
 '신디로퍼',
 '힐링인더시티나잇',
 '가위바위보',
 '따뜻해지는곡',
 '우리아이자장가태교클래식',
 '멍때리며',
 'the_quiett',
 '가을가을해',
 '옹성우노래잘해',
 '기타멘',
 '나알아',
 '그릴요리',
 '힙알못',
 '텐션업',
 '피부샵같은',
 '위시리스트',
 '12곡',
 '신인걸그룹',
 '힐링음악',
 '복',
 '가이세바스찬',
 '아이돌유닛',
 '아트무비',
 '본시리즈',
 '전반기',
 '미안',
 '민욱',
 '투명',
 '봄감성송',
 '가뿐하게',
 'horaceparlan',
 '하드힙합',
 '늦은',
 '요가에어울리는asmr',
 '일요일에',
 'xxlfreshmen',
 '저녁밥',
 '마스터피스',
 'lofihiphop',
 'dprlive',
 '메종데부지',
 'dm뮤직',
 '돈디아블로',
 '그루비',
 'soundgarden',
 '수면곡',
 '가창노래',
 '음악공유',
 '트롯트신',
 '내가카페사장이면이노래튼다',
 '급추위',
 '오프온오프',
 '유은성',
 '핫함',
 'brostep',
 '흔들어라',
 '어쩌면우울',
 '강한비트',
 '유산슬',
 '듀엣감성곡',
 '월드클럽돔2018',
 '벡야',
 '달달한_노래',
 '복싱',
 '비오는날명곡',
 '롯데응원',
 '원산지가고싶다',
 '감미로운_자장가',
 '내_플레이리스트속_인디음악',
 '스산함',
 '아메리카노',
 '얼터알앤비',
 '여름음악회',
 '댄스뮤직',
 '마구마구',
 '이불속에',
 '애니극장판',
 '00년생',
 '노르웨이',
 'woogie',
 '안드레아모티스',
 '가을노래들',
 '천용성',
 '시대별차트',
 'lovepoem',
 '음악의안단테',
 '들려오는',
 '산뜻함',
 '한국락',
 '다가올2018년',
 '좋아하는마음은',
 '화양연화',
 '윤석철',
 '치킨',
 '시카고소울',
 'umfkorea2019',
 '이중창',
 '독일록',
 '김광진

## valid data 불러오기

In [63]:
import copy
valid_data = pd.read_json('data/val.json', encoding='UTF-8', typ = 'frame')

In [64]:
prac = copy.deepcopy(valid_data[['tags','plylst_title','songs','updt_date']])
prac.head()

,tags,plylst_title,songs,updt_date
0,[],,"[373313, 151080, 275346, 696876, 165237, 52593...",2019-05-27 14:14:33.000
1,[],앨리스테이블,[],2014-07-16 15:24:24.000
2,[],,"[529437, 516103, 360067, 705713, 226062, 37089...",2008-06-21 23:26:22.000
3,[],,"[589668, 21711, 570151, 320043, 13930, 599327,...",2017-10-30 18:15:43.000
4,[],,"[672718, 121924, 102694, 683657, 201558, 38511...",2017-02-07 11:40:42.000


### plylst_title부분 전처리하기

In [65]:
def clean_str(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]' # 특수기호 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = "\u3000\u3000"     # 유니코드 삭제
    text = re.sub(pattern=pattern, repl=' ',string=text)
    pattern = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE) #마지막점검 제거
    text = re.sub(pattern=pattern, repl=' ',string=text)
    return text

In [66]:
import re
for i in range(0,len(prac)):
    mylist = [[prac['plylst_title'][i]]]
    clean = clean_str(' '.join(sum(mylist,[])))
    if len(clean)==0:
        clean=' '
    prac['plylst_title'][i] = clean
prac['plylst_title']

0              
1        앨리스테이블
2              
3              
4              
          ...  
23010          
23011          
23012          
23013          
23014          
Name: plylst_title, Length: 23015, dtype: object

In [67]:
for i in range(0,len(prac)):
    if len(prac['plylst_title'][i])==0:
        print('yes')

In [68]:
for i in range(len(prac['plylst_title'])):
    prac['plylst_title'][i]=prac['plylst_title'][i].strip()

In [69]:
for_nouns = copy.deepcopy(prac['plylst_title'])

### plylst title 실질형태소 뽑기

In [70]:
from khaiii import KhaiiiApi
api = KhaiiiApi()
from multiprocessing import Pool
import time

def do_work(i):
    tmp_list = []
    
    if len(prac['plylst_title'][i]) == prac['plylst_title'][i].count(' '):
        # 빈칸으로만 되어있는 행
        pass
    elif prac['plylst_title'][i].find(" ")==-1:
        # 띄어쓰기 없는 행
        tmp_list.append(prac['plylst_title'][i])
    else:
        for word in api.analyze(prac['plylst_title'][i]):
            for morph in word.morphs:
                # 실질형태소 가져오기
                if morph.tag in ['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ','SN','SL']:
                    tmp_list.append(morph.lex)
    if i%100==0:
        print(i)
    if i%1000==0:
        time.sleep(5)
        
    return tmp_list



if __name__ == '__main__':
    start_time = time.time()
    with Pool(processes=4) as p:
        result = p.map(do_work, range(0,len(prac['plylst_title'])))
    print("--- %s seconds ---" % (time.time()-start_time))

02900
1500
4400

3000
4500
1600
1700
4600
1800
4700
19004800

4900
2000
5000
100
200
3100
300
3200400

3300
500
21003400
5100

6002200

35005200

2300700

36005300

8002400
3700

5400
9002500

38005500
1000

2600
3900
5600
27004000

5700
2800
5800
7200
5900
7300
6000
7400
7500
7600
7700
7800
7900
8000
1100
1200
1300
4100
1400
4200
87006100

4300
88006200

10100
63008900

10200
90006400

10300
8100
650010400

8200
105006600

8300
106006700

8400
107006800

8500
10800
6900
8600
10900
7000
11600
11000
11700
11800
11900
12000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
7100
11100
13000
11200
11300
11400
12100
11500
1220014400

1450012300

1460012400

12500
14700
1480012600

1270014900

15000
12800
12900
15900
16000
17300
17400
17500
17600
17700
17800
13100
17900
13200
18000
13300
13400
13500
13600
13700
13800
13900
14000
15100
15200
15300
1540016100

1550016200

16300
15600
16400
15700
16500
15800
1880016600

1890016700

19000
16800
16900
17000
18100
18200
18300
18400
18500
18600
18

In [71]:
### 영어 소문자로
for i in range(0,len(result)):
    for j in range(0, len(result[i])):
        result[i][j] = result[i][j].lower()

### 영어 한글자 지우기
import re
p = re.compile('[a-z]')

for i in range(0,len(result)):
    for j in range(0, len(result[i])):
        if len(result[i][j]) == 1:
            if p.match(result[i][j]) != None:
                result[i][j] = ''

for i in range(0,len(result)):
    while '' in result[i]:
        result[i].remove('')

### 중복되는 단어 제거
for i in range(0,len(result)):
    result[i] = list(set(result[i]))
result


### 숫자 4자리 제외 제거
for i in range(0,len(result)):
    for j in range(0, len(result[i])):
        if result[i][j].isdigit() == True:
            if len(result[i][j]) != 4:
                result[i][j] = ''
for i in range(0,len(result)):
    while '' in result[i]:
        result[i].remove('')

In [72]:
prac['plylst_title']=copy.deepcopy(result)

### 명사 nouns뽑기

In [73]:
from khaiii import KhaiiiApi
api = KhaiiiApi()
from multiprocessing import Pool
import time

def do_work(i):
    tmp_list = []
    
    if len(for_nouns[i]) == for_nouns[i].count(' '):
        # 빈칸으로만 되어있는 행
        pass
    elif for_nouns[i].find(" ")==-1:
        # 띄어쓰기 없는 행
        tmp_list.append(for_nouns[i])
    else:
        for word in api.analyze(for_nouns[i]):
            for morph in word.morphs:
                # 실질형태소 가져오기
                if morph.tag in ['NNG', 'NNP', 'NP','SL']:
                    tmp_list.append(morph.lex)
    if i%100==0:
        print(i)
    if i%1000==0:
        time.sleep(5)
        
    return tmp_list



if __name__ == '__main__':
    start_time = time.time()
    with Pool(processes=4) as p:
        noun_tag = p.map(do_work, range(0,len(for_nouns)))
    print("--- %s seconds ---" % (time.time()-start_time))

0
290044001500


450030001600


17004600

1800
4700
1900
4800
2000
4900
5000
100
200
300
3100
400
3200
500
3300
600
3400
700
3500
800
36002100

90051003700


220010005200


3800
2300
5300
3900
2400
5400
4000
2500
5500
5600
2600
5700
27005800

2800
5900
7200
6000
7300
7400
7500
7600
7700
7800
7900
8000
1100
1200
1300
1400
8700
4100
8800
4200
8900
4300
101009000

10200
6100
10300
6200
10400
6300
10500
6400
10600
6500
10700
660010800

8100
6700
8200
10900
6800
8300
11000
6900
8400
7000
8500
8600
11600
11700
11800
11900
12000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
11100
11200
11300
11400
7100
11500
1300014400

14500
14600
14700
12100
14800
1220014900

15000
12300
12400
12500
12600
12700
12800
12900
15900
16000
17300
17400
17500
17600
1770013100

1780013200

179001510013300


152001800013400


15300
13500
1540013600

1610015500
13700

1560016200
13800

15700
1630013900
15800
16400

1400018800
16500

1890016600

1900016700

16800
16900
17000
18100
18200
18300
18400
18500
18600
18

In [74]:
### 영어 소문자로
for i in range(0,len(noun_tag)):
    for j in range(0, len(noun_tag[i])):
        noun_tag[i][j] = noun_tag[i][j].lower()

### 영어 한글자 지우기
import re
p = re.compile('[a-z]')

for i in range(0,len(noun_tag)):
    for j in range(0, len(noun_tag[i])):
        if len(noun_tag[i][j]) == 1:
            if p.match(noun_tag[i][j]) != None:
                noun_tag[i][j] = ''

for i in range(0,len(noun_tag)):
    while '' in noun_tag[i]:
        noun_tag[i].remove('')

### 중복되는 단어 제거
for i in range(0,len(noun_tag)):
    noun_tag[i] = list(set(noun_tag[i]))

In [75]:
prac['nouns']=copy.deepcopy(noun_tag)
pred_tag=[]
pred_song=[]
for i in range(len(prac)):
    pred_tag.append([])
for i in range(len(prac)):
    pred_song.append([])
prac['pred_song']=pred_song
prac['pred_tag']=pred_tag
prac['index'] = range(len(prac))
prac = prac[['index','tags','plylst_title','nouns','songs','updt_date','pred_tag','pred_song']]
prac

,index,tags,plylst_title,nouns,songs,updt_date,pred_tag,pred_song
0,0,[],[],[],"[373313, 151080, 275346, 696876, 165237, 52593...",2019-05-27 14:14:33.000,[],[]
1,1,[],[앨리스테이블],[앨리스테이블],[],2014-07-16 15:24:24.000,[],[]
2,2,[],[],[],"[529437, 516103, 360067, 705713, 226062, 37089...",2008-06-21 23:26:22.000,[],[]
3,3,[],[],[],"[589668, 21711, 570151, 320043, 13930, 599327,...",2017-10-30 18:15:43.000,[],[]
4,4,[],[],[],"[672718, 121924, 102694, 683657, 201558, 38511...",2017-02-07 11:40:42.000,[],[]
...,...,...,...,...,...,...,...,...
23010,23010,[잔잔한],[],[],"[75842, 26083, 244183, 684715, 500593, 508608,...",2015-12-17 14:06:05.000,[],[]
23011,23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",[],[],"[450275, 487671, 561031, 663944, 628672, 59121...",2020-04-16 21:35:44.000,[],[]
23012,23012,[],[],[],"[625875, 464051, 11657, 236393, 358186, 213435...",2019-03-27 15:27:40.000,[],[]
23013,23013,[],[],[],"[161094, 665833, 688145, 432735, 439938, 12665...",2015-11-18 11:49:09.000,[],[]


In [76]:
# 플레이 제목만 있는 경우
only_play=[]
for i in range(0,len(prac)):
    if len(prac.iloc[i,1]) == 0 and len(prac.iloc[i,2]) !=0 and len(prac.iloc[i,4])==0:
        only_play.append(prac.iloc[i,:].tolist())
len(only_play)

1723

In [77]:
# tag와 플레이제목만 있는 경우
tag_play=[]
for i in range(0,len(prac)):
    if len(prac.iloc[i,1]) != 0 and len(prac.iloc[i,2]) !=0 and len(prac.iloc[i,4])==0:
        tag_play.append(prac.iloc[i,:].tolist())
len(tag_play)

2618

In [78]:
# tag만 있는 경우
only_tag=[]
for i in range(0,len(prac)):
    if len(prac.iloc[i,1]) != 0 and len(prac.iloc[i,2]) ==0 and len(prac.iloc[i,4])==0:
        only_tag.append(prac.iloc[i,:].tolist())
len(only_tag)

12

In [79]:
# tag와 song만 있는 경우
tag_song=[]
for i in range(0,len(prac)):
    if len(prac.iloc[i,1]) != 0 and len(prac.iloc[i,2]) ==0 and len(prac.iloc[i,4])!=0:
        tag_song.append(prac.iloc[i,:].tolist())
len(tag_song)

8975

In [80]:
# song만 있는 경우
only_song=[]
for i in range(0,len(prac)):
    if len(prac.iloc[i,1]) == 0 and len(prac.iloc[i,2]) ==0 and len(prac.iloc[i,4])!=0:
        only_song.append(prac.iloc[i,:].tolist())
len(only_song)

9661

In [81]:
# 아예 없는 경우
nothing=[]
for i in range(0,len(prac)):
    if len(prac.iloc[i,1]) == 0 and len(prac.iloc[i,2]) ==0 and len(prac.iloc[i,4])==0:
        nothing.append(prac.iloc[i,:].tolist())
len(nothing)

26

In [82]:
len(only_play)+len(tag_play)+len(only_tag)+len(tag_song)+len(only_song)+len(nothing) == len(valid_data)

True

## 1. playlist만 있는 경우
- tag 예측: word2vec


- playlist title의 명사 => 유사도에서 unique tag목록에 있는 애들 => 유사도
- 위의 순으로 tag를 채울 것임


- unique tag목록을 먼저 만들기
- word2vec을 하여 모델 만들기
- playlist title의 명사를 khaiii를 통해 뽑아내기

In [33]:
only_play

[[1, [], '앨리스테이블', ['앨리스테이블'], [], [], []],
 [9, [], '기분 좋은 재즈와 함께 만드는 달달한 하루', ['달', '재즈', '기분', '하루'], [], [], []],
 [35, [], '사랑 그리고이별', ['이별', '사랑'], [], [], []],
 [57, [], '마쉬멜로우같은 멜로우한 음악', ['멜로우한', '음악', '마쉬', '우', '멜로'], [], [], []],
 [71, [], '공부와 독서를 위한  Newage', ['독서', 'newage', '공부'], [], [], []],
 [101, [], '새벽에 감성 돋는 노래', ['새벽', '감성', '노래'], [], [], []],
 [115, [], '인디  OST 음악을 만나다 50', ['인디', '음악', 'ost'], [], [], []],
 [135, [], '너 없이 살수가 없어 그리움', ['너', '그리움', '살수'], [], [], []],
 [167, [], '귀를 달달하게 만들어주는 남녀 듀엣곡', ['곡', '남녀', '달달', '듀엣', '귀'], [], [], []],
 [199, [], '기분전환 K POP 뮤직', ['뮤직', '기분', '전환', 'pop'], [], [], []],
 [218, [], '냉정과 열정사이  가을', ['냉정', '열정사이', '가을'], [], [], []],
 [220,
  [],
  '나른한 오후 카페에 앉아 커피를 마시며 듣는 연주곡',
  ['카페', '오후', '연주곡', '커피'],
  [],
  [],
  []],
 [221, [], '발라드드', ['발라드드'], [], [], []],
 [240,
  [],
  '후기 바로크 11 17세기후 18세기중  바흐4',
  ['후기', '바흐', '바로크', '세기', '세기후'],
  [],
  [],
  []],
 [253, [], '신나는 곡 좋아하는 곡', ['곡', '신'], [], [], []],
 [

## 2. tag와 playlist만 있는 경우

In [48]:
tag_play

[[8, ['스트레스'], '리듬타면서 빡시게 운동하자    스트레스 날리자', ['스트레스', '날', '리', '운동'], []],
 [17, ['생각나'], '다시 생각나는 그 사람', ['사람'], []],
 [20, ['피아노', '이루마', '메로디'], '불면증엔 아름다운 뉴에이지 곡들', ['불면증', '지', '뉴에이'], []],
 [40,
  ['발렌타인데이', '달달한', '사랑노래', '고백'],
  '발렌타인 데이  달콤한 초콜릿 매장 음악',
  ['매장', '데이', '초콜릿', '음악', '발렌타'],
  []],
 [42, ['새벽'], '하루 끝 잔잔한 노래로 기분 좋게 잠에 들기', ['잠', '끝', '하루', '노래', '기분'], []],
 [43,
  ['테일러스위프트', '팝의여왕', '컨트리팝'],
  '테일러 스위프트 전곡 모음',
  ['모음', '전곡', '트', '테일러', '스위', '프'],
  []],
 [45, ['감성'], '한국R B I 그루비한  트렌디한  느낌있는', ['트', '느낌', '한국', '그루', '비'], []],
 [49, ['밤', '락', '분위기', '신나는'], '차에서ㅤㅤ', ['차에서ㅤㅤ'], []],
 [55,
  ['댄스', '신남', '신나는'],
  '팝 일렉 신나는 분위기를 원할 때',
  ['때', '일렉', '팝', '신', '분위기'],
  []],
 [56,
  ['비', '달달한', '음색'],
  '달콤한 음색으로 조용히 귓가에 폭 안기는 노래들',
  ['폭', '음색', '노래', '귓가'],
  []],
 [58,
  ['사랑랩', '비트', '인디'],
  '감성적인가사의랩 발라드랩 사랑랩 재즈 알앤비랩',
  ['발라드랩', '가사의랩', '알앤비랩', '사랑랩', '감성', '재즈'],
  []],
 [69, ['연애'], '내 여자에게 들려주고싶은노래', ['여자', '노래', '나'], []],
 [74, ['밤'], '신나는 음악이

## 3. tag만 있는 경우

In [56]:
only_tag

[[160, ['공짜', '광고ost'], '', [], []],
 [1367, ['이별', '회상'], '', [], []],
 [2194, ['휴식', '새벽'], '', [], []],
 [4514, ['알앤비'], '', [], []],
 [16696, ['소울'], '', [], []],
 [19221, ['봄', '연주곡', 'Morning', 'Instrumental', '아침'], '', [], []],
 [22555, ['슬픔', '추억'], '', [], []]]

## 4. tag와 song만 있는 경우

In [58]:
tag_song

[[5,
  ['비', '분위기', '컨트리', '서정적인'],
  '',
  [],
  [173033,
   138237,
   63140,
   15733,
   461780,
   308519,
   645777,
   225090,
   73748,
   204150]],
 [11,
  ['아침', '눈', '크리스마스'],
  '',
  [],
  [174132,
   257367,
   539802,
   568691,
   296891,
   384718,
   367173,
   353844,
   327260,
   617514,
   549031,
   205884,
   344666,
   211309,
   587112,
   349494,
   594472,
   290695]],
 [12,
  ['EDM'],
  '',
  [],
  [392257,
   668879,
   460746,
   680240,
   490966,
   250108,
   431910,
   65548,
   26682,
   361097,
   82191,
   348201,
   547681,
   286564,
   118525,
   155043,
   61432,
   158581,
   228396]],
 [21,
  ['상큼한', '출근길', '트렌디', '팝', '신나는'],
  '',
  [],
  [534729,
   149593,
   677181,
   179617,
   408079,
   34339,
   542189,
   357079,
   191158,
   202182,
   656843,
   500288,
   688895,
   529198,
   26763,
   424707]],
 [23,
  ['발렌타인데이'],
  '',
  [],
  [498353,
   283397,
   588606,
   499171,
   191963,
   462587,
   506285,
   44394,
   236685,
   5

## 5. song만 있는 경우

In [60]:
only_song

[[0,
  [],
  '',
  [],
  [373313,
   151080,
   275346,
   696876,
   165237,
   525935,
   457812,
   371709,
   170292,
   438915,
   162569,
   161136,
   310892,
   608704,
   295701,
   127698,
   269427,
   592526,
   110696,
   368214,
   24386,
   104455,
   61612,
   151421,
   416023,
   320447,
   19606]],
 [2,
  [],
  '',
  [],
  [529437,
   516103,
   360067,
   705713,
   226062,
   370891,
   344934,
   495061,
   297884,
   180274,
   614059,
   175436,
   110325,
   314668]],
 [3,
  [],
  '',
  [],
  [589668,
   21711,
   570151,
   320043,
   13930,
   599327,
   216696,
   423594,
   170279,
   93151,
   455185,
   634366,
   277910,
   671805,
   622636,
   232676,
   442352]],
 [4,
  [],
  '',
  [],
  [672718, 121924, 102694, 683657, 201558, 385110, 610734, 83940]],
 [6, [], '', [], [227044, 424672, 515592, 22408, 57893, 153714, 212469]],
 [7,
  [],
  '',
  [],
  [54457,
   423722,
   337682,
   91214,
   644980,
   556444,
   117928,
   411444,
   148421,
   92569

## 6. 아예 없는 경우

In [62]:
nothing

[[2443, [], '', [], []],
 [3246, [], '', [], []],
 [4521, [], '', [], []],
 [5721, [], '', [], []],
 [6113, [], '', [], []],
 [6661, [], '', [], []],
 [7955, [], '', [], []],
 [8916, [], '', [], []],
 [9479, [], '', [], []],
 [10955, [], '', [], []],
 [12365, [], '', [], []],
 [12607, [], '', [], []],
 [12673, [], '', [], []],
 [13154, [], '', [], []],
 [15867, [], '', [], []],
 [16793, [], '', [], []],
 [17558, [], '', [], []],
 [18551, [], '', [], []],
 [21240, [], '', [], []],
 [22045, [], '', [], []],
 [22830, [], '', [], []]]